<a href="https://colab.research.google.com/github/cipB14/Questify/blob/main/box1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU pdf4llm pymupdf gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00


In [2]:
import pdf4llm
import fitz  # PyMuPDF
import gradio as gr
from google.colab import files

In [9]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [ ]:
# Install required libraries
!pip install -qU transformers accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1️⃣ Enable 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Use float16 for faster computation
    bnb_4bit_use_double_quant=True,  # Improves efficiency
    bnb_4bit_quant_type="nf4"  # NF4 quantization for better accuracy
)

# 2️⃣ Load Tokenizer & Model with Quantization
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=bnb_config,
    device_map="cuda"  # Automatically assigns model to GPU
)

# 3️⃣ Generate a Response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Send input to GPU
    output = model.generate(**inputs, max_new_tokens=1000)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 4️⃣ Test the Model
prompt = "Explain the process of photosynthesis."
print(generate_response(prompt))


In [ ]:
prompt = "Explain the process of photosynthesis in C code."
print(generate_response(prompt))


In [3]:
# 1️⃣ Upload PDF
uploaded = files.upload()

# Get the uploaded filename
pdf_filename = list(uploaded.keys())[0]

# 2️⃣ Extract Page-wise Markdown
page_data = pdf4llm.to_markdown(pdf_filename, page_chunks=True)

# Store page-wise text in an array
pages_array = [page["text"] for page in page_data]

# 3️⃣ Display Extracted Text (First 500 chars of each page)
for i, page_text in enumerate(pages_array[:5]):  # Show first 5 pages
    print(f"\n🔹 Page {i+1} (First 500 chars):\n")
    print(page_text[:])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def generate_questions(page_text):

    prompt = f'''You are an AI assistant specialized in question generation.
Use the provided context to generate three well-formed questions.
Also include the topic name in the question.


Context:
{page_text}

### Questions:

    '''
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=300, temperature=0.7,pad_token_id=tokenizer.eos_token_id,do_sample=True)
    generated_text=  tokenizer.decode(output[0], skip_special_tokens=True)
    if "### Questions:" in generated_text:
        generated_questions = generated_text.split("### Questions:")[1].strip()
    else:
        generated_questions = generated_text  # Fallback in case the marker is missing

    return generated_questions

# 5️⃣ Generate Questions for Each Page
for i, page_text in enumerate(pages_array[:5]):  # Process first 5 pages
    print(f"\n Page {i+1} Questions:\n")
    print(generate_questions(page_text))